# Spam classfication 

## Importing libraries

In [25]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import re
import time
import fasttext
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional, Reshape
from tensorflow.keras.callbacks import EarlyStopping

# import ReshapeLayer
from tensorflow.keras.layers import Layer

%matplotlib inline 

## Dataset Location

In [26]:
dataset = '../data/spam - spam.csv'

## Convert to dataframe

In [27]:
df = pd.read_csv(dataset)
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


# Replacing Column names

In [28]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df = df.rename(columns={'v1': 'target', 'v2': 'message'})
df.head()

,target,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Duplicate removal

In [29]:
print(f'Number of duplicated rows are', df.duplicated().sum())
print(f'The shape of the dataset is', df.shape)
df[df.duplicated()]


Number of duplicated rows are 409
The shape of the dataset is (5572, 2)


,target,message
102,ham,As per your request 'Melle Melle (Oru Minnamin...
153,ham,As per your request 'Melle Melle (Oru Minnamin...
206,ham,"As I entered my cabin my PA said, '' Happy B'd..."
222,ham,"Sorry, I'll call later"
325,ham,No calls..messages..missed calls
...,...,...
5524,spam,You are awarded a SiPix Digital Camera! call 0...
5535,ham,"I know you are thinkin malaria. But relax, chi..."
5539,ham,Just sleeping..and surfing
5553,ham,Hahaha..use your brain dear


In [30]:
df.drop_duplicates(inplace=True)
print(f'The shape of the dataset after removing duplicate is ', df.shape)

The shape of the dataset after removing duplicate is  (5163, 2)


## Text Preprocessing
- Removing Stopwords
- Removing Punctuations

In [31]:
def cleanText(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    return text    

df['message'] = df['message'].apply(cleanText)
df.head()

,target,message
0,ham,go until jurong point crazy available only ...
1,ham,ok lar joking wif u oni
2,spam,free entry in a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don t think he goes to usf he lives aro...


In [32]:
df['message'] = df['message'].apply(cleanText)

## Embedding

In [33]:
model = '../model/lid.bin'
model = fasttext.load_model(model)

def getEmbedding(Text):
    return model.get_sentence_vector(Text)

In [34]:
df['embedding'] = df['message'].apply(getEmbedding)

## Splitting Dataset

In [35]:
X = np.array(df['embedding'].tolist())
y = df['target']

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Deep Learning

In [36]:
model = Sequential()
model.add(Dense(32, input_dim=16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

print(f'Accuracy score is {accuracy_score(y_test, y_pred)}')

Epoch 1/100
130/130 [==============================] - 1s 2ms/step - loss: 0.4464 - accuracy: 0.8705 - val_loss: 0.3374 - val_accuracy: 0.8877
Epoch 2/100
130/130 [==============================] - 0s 1ms/step - loss: 0.3613 - accuracy: 0.8714 - val_loss: 0.3281 - val_accuracy: 0.8877
Epoch 3/100
130/130 [==============================] - 0s 1ms/step - loss: 0.3467 - accuracy: 0.8714 - val_loss: 0.3157 - val_accuracy: 0.8877
Epoch 4/100
130/130 [==============================] - 0s 1ms/step - loss: 0.3297 - accuracy: 0.8714 - val_loss: 0.2992 - val_accuracy: 0.8877
Epoch 5/100
130/130 [==============================] - 0s 1ms/step - loss: 0.3151 - accuracy: 0.8714 - val_loss: 0.2884 - val_accuracy: 0.8877
Epoch 6/100
130/130 [==============================] - 0s 1ms/step - loss: 0.3032 - accuracy: 0.8714 - val_loss: 0.2774 - val_accuracy: 0.8877
Epoch 7/100
130/130 [==============================] - 0s 1ms/step - loss: 0.2945 - accuracy: 0.8714 - val_loss: 0.2694 - val_accuracy: 0.8877

In [37]:
model = Sequential()
model.add(Dense(32, input_dim=16, activation='relu'))
model.add(Reshape((1, 32)))
model.add(LSTM(16, return_sequences=True))
model.add(LSTM(8, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

print(f'Accuracy score is {accuracy_score(y_test, y_pred)}')

2023-05-29 04:02:09.047973: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 04:02:09.049187: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 04:02:09.050117: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/100


2023-05-29 04:02:09.253966: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 04:02:09.255699: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 04:02:09.256640: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

126/130 [============================>.] - ETA: 0s - loss: 0.5328 - accuracy: 0.8718

2023-05-29 04:02:14.767773: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 04:02:14.770953: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 04:02:14.773827: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

130/130 [==============================] - 7s 16ms/step - loss: 0.5299 - accuracy: 0.8714 - val_loss: 0.3480 - val_accuracy: 0.8877
Epoch 2/100
130/130 [==============================] - 1s 6ms/step - loss: 0.3751 - accuracy: 0.8714 - val_loss: 0.3447 - val_accuracy: 0.8877
Epoch 3/100
130/130 [==============================] - 1s 5ms/step - loss: 0.3695 - accuracy: 0.8714 - val_loss: 0.3425 - val_accuracy: 0.8877
Epoch 4/100
130/130 [==============================] - 1s 5ms/step - loss: 0.3609 - accuracy: 0.8714 - val_loss: 0.3317 - val_accuracy: 0.8877
Epoch 5/100
130/130 [==============================] - 1s 4ms/step - loss: 0.3461 - accuracy: 0.8714 - val_loss: 0.3163 - val_accuracy: 0.8877
Epoch 6/100
130/130 [==============================] - 1s 4ms/step - loss: 0.3285 - accuracy: 0.8714 - val_loss: 0.2993 - val_accuracy: 0.8877
Epoch 7/100
130/130 [==============================] - 1s 5ms/step - loss: 0.3133 - accuracy: 0.8714 - val_loss: 0.2855 - val_accuracy: 0.8877
Epoch 8/10

2023-05-29 04:02:40.497540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 04:02:40.499796: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 04:02:40.502750: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

33/33 [==============================] - 1s 2ms/step
Accuracy score is 0.9002904162633107


## Bag of Words

In [38]:
cv = CountVectorizer()
X = cv.fit_transform(df['message'])
y = df['target']

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [39]:
model = Sequential()

model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train.toarray(), y_train, epochs=100, batch_size=32, validation_data=(X_test.toarray(), y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

y_pred = model.predict(X_test.toarray())
y_pred = np.round(y_pred).astype(int)

print(f'Accuracy score is {accuracy_score(y_test, y_pred)}')

Epoch 1/100
130/130 [==============================] - 2s 7ms/step - loss: 0.3006 - accuracy: 0.9189 - val_loss: 0.1201 - val_accuracy: 0.9797
Epoch 2/100
130/130 [==============================] - 1s 6ms/step - loss: 0.0482 - accuracy: 0.9903 - val_loss: 0.0823 - val_accuracy: 0.9806
Epoch 3/100
130/130 [==============================] - 1s 6ms/step - loss: 0.0157 - accuracy: 0.9969 - val_loss: 0.0959 - val_accuracy: 0.9816
Epoch 4/100
130/130 [==============================] - 1s 5ms/step - loss: 0.0073 - accuracy: 0.9985 - val_loss: 0.0930 - val_accuracy: 0.9855
Epoch 5/100
33/33 [==============================] - 0s 3ms/step
Accuracy score is 0.9864472410454985


In [40]:
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Reshape((1, 32)))
model.add(LSTM(16, return_sequences=True))
model.add(LSTM(8, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train.toarray(), y_train, epochs=100, batch_size=32, validation_data=(X_test.toarray(), y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

y_pred = model.predict(X_test.toarray())
y_pred = np.round(y_pred).astype(int)

print(f'Accuracy score is {accuracy_score(y_test, y_pred)}')

2023-05-29 04:02:47.756840: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 04:02:47.758559: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 04:02:47.759875: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/100


2023-05-29 04:02:48.599289: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 04:02:48.601356: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 04:02:48.605738: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

128/130 [============================>.] - ETA: 0s - loss: 0.4294 - accuracy: 0.9033

2023-05-29 04:02:53.977881: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 04:02:53.980437: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 04:02:53.982637: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

130/130 [==============================] - 6s 15ms/step - loss: 0.4273 - accuracy: 0.9039 - val_loss: 0.1736 - val_accuracy: 0.9739
Epoch 2/100
130/130 [==============================] - 1s 8ms/step - loss: 0.1225 - accuracy: 0.9862 - val_loss: 0.1155 - val_accuracy: 0.9806
Epoch 3/100
130/130 [==============================] - 1s 8ms/step - loss: 0.0696 - accuracy: 0.9944 - val_loss: 0.0913 - val_accuracy: 0.9845
Epoch 4/100
130/130 [==============================] - 1s 7ms/step - loss: 0.0351 - accuracy: 0.9969 - val_loss: 0.0832 - val_accuracy: 0.9806
Epoch 5/100
130/130 [==============================] - 1s 7ms/step - loss: 0.0200 - accuracy: 0.9981 - val_loss: 0.0845 - val_accuracy: 0.9787
Epoch 6/100
130/130 [==============================] - 1s 8ms/step - loss: 0.0142 - accuracy: 0.9985 - val_loss: 0.0863 - val_accuracy: 0.9787
Epoch 7/100
130/130 [==============================] - 1s 7ms/step - loss: 0.0117 - accuracy: 0.9985 - val_loss: 0.0894 - val_accuracy: 0.9797


2023-05-29 04:03:00.889001: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 04:03:00.890886: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 04:03:00.892348: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

33/33 [==============================] - 1s 3ms/step
Accuracy score is 0.9796708615682478
